# Upload WikiAnswers_small

In [2]:
from datasets import load_dataset

dataset = load_dataset("embedding-data/WikiAnswers", cache_dir="wikianswers/")

c:\Users\nikhi\Documents\Projects\WikiAnswers Embeddings\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
small_dataset = dataset["train"].train_test_split(train_size=0.04, seed=42)

In [7]:
del small_dataset["test"]
small_dataset

DatasetDict({
    train: Dataset({
        features: ['set'],
        num_rows: 1095326
    })
})

In [8]:
small_dataset.save_to_disk("wikianswers_small/")

Saving the dataset (4/4 shards): 100%|██████████| 1095326/1095326 [05:58<00:00, 3052.19 examples/s]


In [9]:
dataset_name = "nikhilchigali/wikianswers_small"
small_dataset.push_to_hub(dataset_name)

Uploading the dataset shards: 100%|██████████| 4/4 [10:55<00:00, 164.00s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/nikhilchigali/wikianswers_small/commit/6a2d404029aeb4e4f98f3de5000bd9bbad3299c4', commit_message='Upload dataset', commit_description='', oid='6a2d404029aeb4e4f98f3de5000bd9bbad3299c4', pr_url=None, pr_revision=None, pr_num=None)

# Adding new column

In [2]:
from datasets import load_dataset

dataset = load_dataset("nikhilchigali/wikianswers_small", cache_dir="wikianswers_small/")
dataset

Generating train split:   0%|          | 0/30472041 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'cluster'],
        num_rows: 30472041
    })
})

In [51]:
from sentence_transformers import SentenceTransformer


num_rows = len(dataset["train"])
with open("wikianswers_small.csv", "w" ,encoding="utf-8") as f :
    for i, item in enumerate(dataset["train"]):
        print(f"Cluster number: {i}/{num_rows}", end="\r")
        for s in item["set"]:
            f.write(f"{s},{i}\n")

In [ ]:
from datasets import Dataset

def gen():
    for i, item in enumerate(dataset["train"]):
            print(f"Cluster number: {i}/{num_rows}", end="\r")
            for s in item["set"]:
                yield {"sentence": s, "cluster": i}

dataset = Dataset.from_generator(gen)

In [56]:
dataset.save_to_disk("wikianswers_small/")
dataset.push_to_hub("nikhilchigali/wikianswers_small")

Uploading the dataset shards: 100%|██████████| 4/4 [00:17<00:00,  4.35s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/nikhilchigali/wikianswers_small/commit/4e5d4d3a9a9f3775d845303dc7e73ccc7de76b30', commit_message='Upload dataset', commit_description='', oid='4e5d4d3a9a9f3775d845303dc7e73ccc7de76b30', pr_url=None, pr_revision=None, pr_num=None)

In [57]:
ds = load_dataset("nikhilchigali/wikianswers_small", cache_dir="wikianswers_small/")
ds

Generating train split: 100%|██████████| 30472041/30472041 [00:10<00:00, 2997824.00 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'cluster'],
        num_rows: 30472041
    })
})

# Test Embedding creation

In [22]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

model = SentenceTransformer("all-MiniLM-L6-v2")
dataset = load_dataset("nikhilchigali/wikianswers_small", cache_dir="wikianswers_small/")
dataset = dataset["train"].train_test_split(train_size=0.001, seed=42)
del dataset["test"]

print(len(dataset["train"]))


30472


In [23]:
from functools import partial

def embed(model, item):
    return {
            "sentence": item["sentence"],
            "cluster": item["cluster"],
            "embedding_384": model.encode(item["sentence"]),
        }

embed_func = partial(embed, model)

embedded_dataset = dataset.map(
        embed_func,
        batched=True,
        batch_size=128,
        num_proc=4,
    )

Map (num_proc=4):   0%|          | 0/30472 [00:00<?, ? examples/s]

In [27]:
embedded_dataset, embedded_dataset["train"][:5]

(DatasetDict({
     train: Dataset({
         features: ['sentence', 'cluster', 'embedding_384'],
         num_rows: 30472
     })
 }),
 {'sentence': ['Difference between control and controlling?',
   'What is the time difference from California to florida?',
   'What is Lady Mcbeths signal that she has taken care of the guards?',
   'Where does da vinci live in assassins creed?',
   'How long does it take to become a system enginer?'],
  'cluster': [920930, 939842, 953248, 234909, 655386],
  'embedding_384': [[0.003050488419830799,
    -0.04910978302359581,
    -0.06662604957818985,
    0.09611787647008896,
    -0.041146859526634216,
    0.06693094968795776,
    0.04414232075214386,
    0.019050462171435356,
    0.0613543875515461,
    0.09777785837650299,
    0.11683674901723862,
    0.01016086433082819,
    0.031497739255428314,
    -0.0982917919754982,
    -0.01471031829714775,
    0.014941313304007053,
    0.04488522186875343,
    -0.04545489326119423,
    -0.06133206933736801,
  